In [44]:
# set environment

import pandas as pd
import numpy as np
import re


In [45]:
# read SNAP and NSC data

nsc2010 = pd.read_csv('zaf-nsce-2010-centre-overall-v1.0.csv', encoding='ISO-8859-1')
nsc2011 = pd.read_csv('zaf-nsce-2011-centre-overall-v1.0.csv', encoding='ISO-8859-1')
masterlist = pd.read_csv('snaps-1997-2013-masterlist-v1.csv', encoding='ISO-8859-1')
geoinfo = pd.read_csv('district.csv', encoding='ISO-8859-1')

print('original data size:', nsc2010.shape, nsc2011.shape, masterlist.shape, geoinfo.shape)


# drop SNAP and NSC irrelevant variables columns

masterlist = masterlist[['natemis','datayear','schoolname','phase_doe','eidistrict','quintile','sector']]
nsc2010 = nsc2010[['date','emisno','gender','province','regionname','districtname','achieved']]
nsc2011 = nsc2011[['date','emisno','gender','province','regionname','districtname','quintile','achieved']]

print('keep relevant columns:', nsc2010.shape, nsc2011.shape, masterlist.shape)


# drop SNAP and NSC irrelevant observations rows

masterlist = masterlist[ (masterlist.sector=='1') & ((masterlist.phase_doe=='4') | (masterlist.phase_doe=='5')) ]
nsc2010 = nsc2010[ (nsc2010.achieved.notna()) & (nsc2010.emisno!=0) ]
nsc2011 = nsc2011[ (nsc2011.achieved.notna()) & (nsc2011.emisno!=0) ]

print('keep relevant rows:', nsc2010.shape, nsc2011.shape, masterlist.shape)


original data size: (12932, 16) (13048, 17) (433764, 12) (79, 7)
keep relevant columns: (12932, 7) (13048, 8) (433764, 7)
keep relevant rows: (12872, 7) (12919, 8) (150357, 7)


In [46]:
# calculate NSC school achieved rate

def passrate(nscyear):

    # average school passrate for genders
    passrate = nscyear.groupby('emisno')['achieved'].mean()
    nscyear['achieved'][nscyear.duplicated('emisno')] = passrate

    # keep unique passrate for each emis school
    nscyear = nscyear.drop_duplicates('emisno')
    nscyear = nscyear.drop(['gender'], axis=1)
    
    return nscyear

nsc2010 = passrate(nsc2010)
nsc2011 = passrate(nsc2011)

print('NSC data observations:', nsc2010.shape[0], nsc2011.shape[0])


NSC data observations: 6503 6519


In [47]:
# add quintile to NSC2010 from NSC2011

nsc2010 = nsc2010.merge(nsc2011[['emisno','quintile']], on=['emisno'], how='left')

print('NSC quintile size_1:', nsc2010.groupby('quintile').size(), nsc2011.groupby('quintile').size())


# add quintile to NSC2010 NSC2011 from SNAP

def quintile(nscyear, snapdata):

    # emis in NSC 2010 OR 2011
    nscemis = pd.concat([nsc2010.emisno, nsc2011.emisno], ignore_index=True).unique()

    # keep SNAP data with relevant NSC emis
    snapdata = snapdata[snapdata.natemis.isin(nscemis)]

    # keep unique(smallest) quintile for each emis in SNAP
    snapdata = snapdata.sort_values('quintile').drop_duplicates('natemis')

    # merge SNAP and NSC quintile
    snapdata = snapdata.rename(columns={'natemis':'emisno', 'quintile':'quintile_snap'})
    nsc_with_q = pd.merge(nscyear, snapdata[['emisno','quintile_snap','eidistrict']], on=['emisno'], how='left')

    # replace NSC invalid quintile from SNAP
    nsc_with_q.quintile = np.where(nsc_with_q.quintile.isin([1,2,3,4,5]), nsc_with_q.quintile, nsc_with_q.quintile_snap)
    nsc_with_q = nsc_with_q.drop(['quintile_snap'], axis=1)

    # set NSC invalid quintile to 99
    nsc_with_q.quintile = np.where(nsc_with_q.quintile.isin([1,2,3,4,5]), nsc_with_q.quintile, 99)

    return nsc_with_q, snapdata

nsc2010, snap = quintile(nsc2010, masterlist)
nsc2011, masterlist = quintile(nsc2011, masterlist)

print('NSC quintile size_2:', nsc2010.groupby('quintile').size(), nsc2011.groupby('quintile').size())


NSC quintile size_1: quintile
0.0      773
1.0     1494
2.0     1268
3.0     1391
4.0      606
5.0      607
99.0     307
dtype: int64 quintile
0      806
1     1502
2     1278
3     1398
4      607
5      607
99     321
dtype: int64
NSC quintile size_2: quintile
1.0     1555
2.0     1315
3.0     1412
4.0      616
5.0      611
99.0     994
dtype: int64 quintile
1.0     1559
2.0     1323
3.0     1417
4.0      613
5.0      611
99.0     996
dtype: int64


In [48]:
# add district to NSC2010 NSC2011

def district(nscyear):

    # select non-string value in NSC districtname
    nonstr = nscyear.districtname.str.findall(r'\d+').explode().unique()

    # replace NSC non-string districtname from SNAP
    nscyear.districtname = np.where(nscyear.districtname.isin(nonstr), nscyear.eidistrict, nscyear.districtname)
    
    # set NSC invalid districtname to 99
    nscyear.districtname.replace(np.nan, 99, inplace=True)

    # add code by mapping NSC district name
    df_district = nscyear[nscyear.districtname.isin(geoinfo.districtname)]
    merge_district = df_district.merge(geoinfo[['districtname','code','provincecode']], on=['districtname'], how='left')

    # add code by mapping NSC region name
    df_region = nscyear.drop(df_district.index)
    merge_region = df_region.merge(geoinfo[['regionname','code','provincecode']], on=['regionname'], how='left')
    merge_region.drop_duplicates('emisno', inplace=True)

    # combine to creat final NSC data
    nsc_with_district = pd.concat([merge_district, merge_region], ignore_index=True)
    nsc_with_district = nsc_with_district.drop(['regionname','districtname','eidistrict','province'], axis=1)

    return nsc_with_district

nsc2010 = district(nsc2010)
nsc2011 = district(nsc2011)

# check NSC district school number

print('NSC data size:', nsc2010.shape, nsc2011.shape)
print('NSC district size:', nsc2010.groupby('provincecode').size(), nsc2011.groupby('provincecode').size())


NSC data size: (6503, 6) (6519, 6)
NSC district size: provincecode
1     661
2     658
3     214
4     236
5    1682
6     376
7     742
8     518
9    1416
dtype: int64 provincecode
1     661
2     663
3     213
4     238
5    1692
6     379
7     737
8     405
9    1531
dtype: int64
